In [22]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [3]:
#index_col specifies the first column is just an index column
matches=pd.read_csv("matches.csv",index_col=0)

In [6]:
matches.shape

(1032, 27)

In [7]:
matches['team'].value_counts()

team
Liverpool                   52
Newcastle United            52
Burnley                     52
Nottingham Forest           52
Brentford                   52
Everton                     52
Crystal Palace              52
Fulham                      52
Manchester City             52
Sheffield United            52
Wolverhampton Wanderers     52
West Ham United             52
Brighton and Hove Albion    52
Manchester United           52
Aston Villa                 52
Arsenal                     52
Bournemouth                 50
Tottenham Hotspur           50
Luton Town                  50
Chelsea                     50
Name: count, dtype: int64

In [11]:
#Nmodel can only work on columns with numerical types and not on object types
matches.dtypes

date            datetime64[ns]
time                    object
comp                    object
round                   object
day                     object
venue                   object
result                  object
gf                     float64
ga                     float64
opponent                object
xg                     float64
xga                    float64
poss                   float64
attendance             float64
captain                 object
formation               object
referee                 object
match report            object
notes                  float64
sh                     float64
sot                    float64
dist                   float64
fk                     float64
pk                       int64
pkatt                    int64
season                   int64
team                    object
dtype: object

In [10]:
matches['date']=pd.to_datetime(matches['date'])

In [12]:
#Venue Code Predictor 
#Convert Home and Away to Numbers
#Code is 0 for away and 1 for home
matches['venue_code']=matches['venue'].astype("category").cat.codes

In [14]:

#Adding a code to each opponenent
matches['opp_code']=matches['opponent'].astype("category").cat.codes

In [15]:
#Removing the minutes from time and only getting the hour in type int
matches['hour']=matches['time'].str.replace(":.+","",regex=True).astype("int")

In [21]:
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkatt,season,team,venue_code,opp_code,hour,day_code,target
0,2023-08-13,16:30,Premier League,Matchweek 1,Sun,Away,D,1.0,1.0,Chelsea,...,0.0,0,0,2024,Liverpool,0,6,16,6,0
1,2023-08-19,15:00,Premier League,Matchweek 2,Sat,Home,W,3.0,1.0,Bournemouth,...,1.0,0,1,2024,Liverpool,1,2,15,5,1
2,2023-08-27,16:30,Premier League,Matchweek 3,Sun,Away,W,2.0,1.0,Newcastle Utd,...,1.0,0,0,2024,Liverpool,0,14,16,6,1
3,2023-09-03,14:00,Premier League,Matchweek 4,Sun,Home,W,3.0,0.0,Aston Villa,...,0.0,0,0,2024,Liverpool,1,1,14,6,1
4,2023-09-16,12:30,Premier League,Matchweek 5,Sat,Away,W,3.0,1.0,Wolves,...,0.0,0,0,2024,Liverpool,0,19,12,5,1


In [18]:
#Assign a Number for Every Day of the Week
#dt day of week gets the day for each date 
matches['day_code']=matches['date'].dt.day_of_week

In [20]:
matches['target']=(matches['result']=="W").astype("int")

In [23]:
#Random Forest is series of descison trees with slightly different parameters
#min 10 samples then split 
#random state of 1 gives us same results as long as data is same 
rf=RandomForestClassifier(n_estimators=50,min_samples_split=10,random_state=1)

In [26]:
train=matches[matches['date']<'2024-01-01']

In [28]:
test=matches[matches['date']>'2024-01-01']

In [31]:
predictors=['venue_code','opp_code','hour','day_code']

In [32]:
#Fit the model using the predictors and find the target
rf.fit(train[predictors],train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [33]:
#Test the model using test data 
preds=rf.predict(test[predictors])

In [34]:
from sklearn.metrics import accuracy_score

In [35]:
#Calculate the accuracy of the test data 
acc=accuracy_score(test['target'],preds)

In [36]:
acc

0.5245901639344263

In [37]:
#Combine to see where probability for predicitons was highest
combined=pd.DataFrame(dict(actual=test['target'],prediction=preds))

In [39]:
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

prediction,0,1
actual,,
0,92,62
1,54,36


In [40]:
from sklearn.metrics import precision_score

In [41]:
precision_score(test["target"], preds)

0.3673469387755102

In [42]:
#Groups the data my teams 
grouped_matches=matches.groupby("team")

In [47]:
#All of Man City Matches grouped by date 
group = grouped_matches.get_group("Manchester City").sort_values("date")

In [50]:
#Computing rolling averages for each team
def rolling_averages(group,cols,new_cols):
    group=group.sort_values("date")
    #Closed=left ignores the current week
    rolling_stats=group[cols].rolling(3,closed='left').mean()
    group[new_cols]=rolling_stats
    group=group.dropna(subset=new_cols)
    return group    

In [44]:
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]

In [45]:
#For new columns add the word rolling to the end of these columns 
new_cols = [f"{c}_rolling" for c in cols]

In [68]:
rolling_averages(group,cols,new_cols)

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1.0,0.0,Newcastle Utd,...,5,1,2.333333,0.000000,16.000000,6.666667,15.233333,0.000000,0.000000,0.000000
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2.0,1.0,Sheffield Utd,...,6,1,1.666667,0.000000,15.000000,5.333333,16.566667,0.000000,0.000000,0.000000
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2.0,1.0,Sheffield Utd,...,6,1,1.333333,0.333333,19.000000,5.666667,17.700000,0.666667,0.000000,0.333333
5,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Home,W,5.0,1.0,Fulham,...,5,1,1.666667,0.666667,24.000000,7.333333,17.500000,1.333333,0.000000,0.666667
5,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Home,W,5.0,1.0,Fulham,...,5,1,3.000000,1.000000,21.333333,7.333333,16.466667,1.333333,0.333333,1.000000
6,2023-09-16,15:00,Premier League,Matchweek 5,Sat,Away,W,3.0,1.0,West Ham,...,5,1,4.000000,1.000000,13.666667,5.666667,15.633333,0.666667,0.666667,1.000000
6,2023-09-16,15:00,Premier League,Matchweek 5,Sat,Away,W,3.0,1.0,West Ham,...,5,1,4.333333,1.000000,13.666667,7.000000,15.333333,0.333333,0.666667,0.666667
8,2023-09-23,15:00,Premier League,Matchweek 6,Sat,Home,W,2.0,0.0,Nott'ham Forest,...,5,1,3.666667,1.000000,21.333333,10.000000,15.866667,0.666667,0.333333,0.333333
8,2023-09-23,15:00,Premier League,Matchweek 6,Sat,Home,W,2.0,0.0,Nott'ham Forest,...,5,1,2.666667,0.666667,21.666667,10.000000,16.666667,1.333333,0.000000,0.000000
10,2023-09-30,15:00,Premier League,Matchweek 7,Sat,Away,L,1.0,2.0,Wolves,...,5,0,2.333333,0.333333,14.333333,7.000000,16.933333,1.666667,0.000000,0.000000


In [69]:
#Apply the same function for all the teams
matches_rolling=matches.groupby('team').apply(lambda x:rolling_averages(x,cols,new_cols))


In [74]:
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
0,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1.0,0.0,Crystal Palace,...,0,1,1.666667,0.666667,14.333333,5.333333,18.200000,0.000000,0.333333,0.333333
1,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2.0,2.0,Fulham,...,5,0,1.333333,0.333333,13.666667,3.666667,17.300000,0.000000,0.666667,0.666667
2,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2.0,2.0,Fulham,...,5,0,1.333333,0.666667,14.666667,4.333333,15.533333,0.000000,1.000000,1.000000
3,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3.0,1.0,Manchester Utd,...,6,1,1.666667,1.333333,16.333333,6.666667,14.666667,0.000000,1.000000,1.000000
4,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3.0,1.0,Manchester Utd,...,6,1,2.333333,1.666667,17.666667,7.666667,14.200000,0.000000,0.666667,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967,2024-02-10,15:00,Premier League,Matchweek 24,Sat,Home,L,0.0,2.0,Brentford,...,5,0,2.666667,2.000000,14.333333,5.666667,16.800000,1.666667,0.666667,0.666667
968,2024-02-17,15:00,Premier League,Matchweek 25,Sat,Away,W,2.0,1.0,Tottenham,...,5,1,1.333333,2.000000,15.666667,5.333333,17.000000,1.333333,0.333333,0.333333
969,2024-02-17,15:00,Premier League,Matchweek 25,Sat,Away,W,2.0,1.0,Tottenham,...,5,1,0.666667,1.666667,15.333333,5.666667,16.766667,0.666667,0.000000,0.000000
970,2024-02-25,13:30,Premier League,Matchweek 26,Sun,Home,W,1.0,0.0,Sheffield Utd,...,6,1,1.333333,1.333333,13.666667,6.333333,16.333333,0.333333,0.000000,0.000000


In [71]:
matches_rolling = matches_rolling.droplevel('team')

In [73]:
#Get unique value for the index column for each row 
matches_rolling.index= range(matches_rolling.shape[0])

In [81]:
def make_predictions(data,predictors):
    train=data[data['date']<'2024-01-01']
    test=data[data['date']>'2024-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    precison = precision_score(test["target"], preds)
    return combined,precison
    


In [82]:
combined,precison = make_predictions(matches_rolling, predictors + new_cols)

In [83]:
precison

0.5340909090909091

In [84]:
combined

,actual,predicted
37,1,1
38,1,0
39,1,0
40,1,1
41,1,1
...,...,...
967,0,1
968,1,0
969,1,0
970,1,0


In [85]:
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)

In [86]:
combined

,actual,predicted,date,team,opponent,result
37,1,1,2024-01-20,Arsenal,Crystal Palace,W
38,1,0,2024-01-20,Arsenal,Crystal Palace,W
39,1,0,2024-01-30,Arsenal,Nott'ham Forest,W
40,1,1,2024-01-30,Arsenal,Nott'ham Forest,W
41,1,1,2024-02-04,Arsenal,Liverpool,W
...,...,...,...,...,...,...
967,0,1,2024-02-10,Wolverhampton Wanderers,Brentford,L
968,1,0,2024-02-17,Wolverhampton Wanderers,Tottenham,W
969,1,0,2024-02-17,Wolverhampton Wanderers,Tottenham,W
970,1,0,2024-02-25,Wolverhampton Wanderers,Sheffield Utd,W


In [87]:
#Some team names are different in oponents and the actual team name 
#Retruns same value if same name in both columns
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {"Brighton and Hove Albion": "Brighton", "Manchester United": "Manchester Utd", "Newcastle United": "Newcastle Utd", "Tottenham Hotspur": "Tottenham", "West Ham United": "West Ham", "Wolverhampton Wanderers": "Wolves"} 
mapping = MissingDict(**map_values)

In [88]:
combined["new_team"] = combined["team"].map(mapping)

In [89]:
combined

,actual,predicted,date,team,opponent,result,new_team
37,1,1,2024-01-20,Arsenal,Crystal Palace,W,Arsenal
38,1,0,2024-01-20,Arsenal,Crystal Palace,W,Arsenal
39,1,0,2024-01-30,Arsenal,Nott'ham Forest,W,Arsenal
40,1,1,2024-01-30,Arsenal,Nott'ham Forest,W,Arsenal
41,1,1,2024-02-04,Arsenal,Liverpool,W,Arsenal
...,...,...,...,...,...,...,...
967,0,1,2024-02-10,Wolverhampton Wanderers,Brentford,L,Wolves
968,1,0,2024-02-17,Wolverhampton Wanderers,Tottenham,W,Wolves
969,1,0,2024-02-17,Wolverhampton Wanderers,Tottenham,W,Wolves
970,1,0,2024-02-25,Wolverhampton Wanderers,Sheffield Utd,W,Wolves


In [90]:
#Merge the data frame with itself so the same match is not shown twice from both sides 
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])

In [91]:
merged

,actual_x,predicted_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,predicted_y,team_y,opponent_y,result_y,new_team_y
0,1,1,2024-01-20,Arsenal,Crystal Palace,W,Arsenal,0,0,Crystal Palace,Arsenal,L,Crystal Palace
1,1,1,2024-01-20,Arsenal,Crystal Palace,W,Arsenal,0,0,Crystal Palace,Arsenal,L,Crystal Palace
2,1,0,2024-01-20,Arsenal,Crystal Palace,W,Arsenal,0,0,Crystal Palace,Arsenal,L,Crystal Palace
3,1,0,2024-01-20,Arsenal,Crystal Palace,W,Arsenal,0,0,Crystal Palace,Arsenal,L,Crystal Palace
4,1,0,2024-01-30,Arsenal,Nott'ham Forest,W,Arsenal,0,0,Nottingham Forest,Arsenal,L,Nottingham Forest
...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,1,0,2024-02-17,Wolverhampton Wanderers,Tottenham,W,Wolves,0,1,Tottenham Hotspur,Wolves,L,Tottenham
436,1,0,2024-02-25,Wolverhampton Wanderers,Sheffield Utd,W,Wolves,0,0,Sheffield United,Wolves,L,Sheffield United
437,1,0,2024-02-25,Wolverhampton Wanderers,Sheffield Utd,W,Wolves,0,0,Sheffield United,Wolves,L,Sheffield United
438,1,0,2024-02-25,Wolverhampton Wanderers,Sheffield Utd,W,Wolves,0,0,Sheffield United,Wolves,L,Sheffield United


In [92]:
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] ==0)]["actual_x"].value_counts()

actual_x
1    72
0    50
Name: count, dtype: int64

In [93]:
72/122

0.5901639344262295